In [5]:
from base import BaseModel 
from lite_llm import LiteLLMInput
from lite_llm import LiteLLMSetting
from lite_llm import LiteLLMService
from lite_llm import Role 
from lite_llm import CompletionMessage
from pydantic import Field
import base64
from pydantic import HttpUrl, SecretStr
from generation.shared.utils import get_previous_lectures
from storage.minio import MinioInput, MinioService, MinioSetting

minio_setting = MinioSetting(
    endpoint="localhost:9000",
    access_key="minioadmin",
    secret_key="minioadmin123",
    secure=False,
)
        
minio_service = MinioService(settings=minio_setting)

litellm_setting = LiteLLMSetting(
    url=HttpUrl("http://localhost:9510"),
    token=SecretStr("abc123"),
    model="gemini-2.5-flash",
    frequency_penalty=0.0,
    n=1,
    temperature=0.0,
    top_p=1.0,
    max_completion_tokens=10000,
    dimension=1536,
    embedding_model="gemini-embedding"
)

litellm_service = LiteLLMService(litellm_setting=litellm_setting)

class Question(BaseModel):
    question: str = Field(..., description="The quiz question")
    answer: str = Field(..., description="The correct answer to the quiz question")
    distractors: list[str] = Field(..., description="A list of incorrect answers to the quiz question")
    explanation: str = Field(..., description="An explanation of the correct answer")
    
class Questions(BaseModel):
    questions: list[Question] = Field(..., description="A list of quiz questions")

system_prompt = """
You are an expert educational content creator and quiz designer. Your task is to create comprehensive and challenging quiz questions based on the provided PDF lecture material and summaries from previous lectures.

Guidelines for creating quiz questions:

1. **Content Coverage**: 
   - Focus primarily on the current PDF lecture content (80% of questions)
   - Include some questions that connect current content with previous lectures (20% of questions)
   - Cover key concepts, definitions, algorithms, formulas, and practical applications

2. **Question Types and Difficulty**:
   - Create questions of varying difficulty levels (30% easy, 50% medium, 20% hard)
   - Include conceptual understanding questions, not just memorization
   - Add application-based questions that test practical knowledge
   - Include questions about advantages/disadvantages of different approaches

3. **Question Structure**:
   - Each question should have exactly 4 answer choices (1 correct + 3 distractors)
   - Distractors should be plausible but clearly incorrect to knowledgeable students
   - Avoid "all of the above" or "none of the above" options
   - Make questions clear and unambiguous

4. **Answer Explanations**:
   - Provide detailed explanations for why the correct answer is right
   - Briefly explain why the distractors are incorrect when helpful
   - Include relevant formulas, concepts, or references to lecture content

5. **Content Integration**:
   - When using previous lecture summaries, create questions that show progression of learning
   - Highlight connections between current and previous topics
   - Ensure questions test understanding rather than rote memorization

6. **Quality Standards**:
   - Questions should be appropriate for university-level students
   - Use precise technical terminology from the field
   - Ensure mathematical notation and formulas are accurate
   - Create questions that encourage critical thinking

Generate 8 high-quality quiz questions following these guidelines.
"""

user_prompt = """
Here is the summary from previous lectures:
{summary}
Please use the pdf file and the summary above to create a quiz for the students including exactly 8 questions.
"""

pdf_path = "/home/vuiem/KLTN/test/files/Lecture 6_Classification_SVM.pdf"

summaries = get_previous_lectures(minio_service, "int3405", 6)

with open(pdf_path, 'rb') as pdf_file:
    pdf_bytes = pdf_file.read()

output = await litellm_service.process_async(
    inputs=LiteLLMInput(
        messages=[
            CompletionMessage(
                role=Role.SYSTEM,
                content=system_prompt
            ),
            CompletionMessage(
                role=Role.USER,
                file_url=f"data:application/pdf;base64,{base64.b64encode(pdf_bytes).decode('utf-8')}"
            ),
            CompletionMessage(
                role=Role.USER,
                content=user_prompt.format(summary="\n".join(summaries))
            )
        ],
        response_format=Questions,
        model="claude-sonnet-4-20250514",
        temperature=0.0,
        max_completion_tokens=10000,
        top_p=1.0,
        frequency_penalty=0.0,
        n=1,
    )
)

output

LiteLLMOutput(response=Questions(questions=[Question(question='What is the primary intuition behind Support Vector Machines (SVM) for finding the optimal decision boundary?', answer='Maximize the margin between the separating hyperplane and the closest data points from both classes', distractors=['Minimize the total number of misclassified training examples', 'Maximize the likelihood of the training data given the model parameters', 'Minimize the entropy of the class distributions at the decision boundary'], explanation="SVM's core intuition is to find the separating hyperplane that maximizes the margin - the width that the boundary could be increased by before hitting a data point. This approach aims to find the decision boundary that is as far as possible from the nearest training examples of both classes, which typically leads to better generalization."), Question(question='In the soft margin SVM formulation, what role does the regularization parameter C play?', answer='It controls 

In [7]:
output.response.questions

[Question(question='What is the primary intuition behind Support Vector Machines (SVM) for finding the optimal decision boundary?', answer='Maximize the margin between the separating hyperplane and the closest data points from both classes', distractors=['Minimize the total number of misclassified training examples', 'Maximize the likelihood of the training data given the model parameters', 'Minimize the entropy of the class distributions at the decision boundary'], explanation="SVM's core intuition is to find the separating hyperplane that maximizes the margin - the width that the boundary could be increased by before hitting a data point. This approach aims to find the decision boundary that is as far as possible from the nearest training examples of both classes, which typically leads to better generalization."),
 Question(question='In the soft margin SVM formulation, what role does the regularization parameter C play?', answer='It controls the trade-off between maximizing the margi

In [8]:
import json
with open("baseline.json", "w") as f:
    json.dump(
        output.response.model_dump(),
        f,
        indent=4,
    )

In [12]:
with open("quiz_generation_test_output_gemini_gpt4omini.json", "r") as f:
    quiz_generation_test_output_gemini_gpt4omini = json.load(f)
 
formatted_questions = {
    "questions": []
}

for i, question in enumerate(quiz_generation_test_output_gemini_gpt4omini["quiz_questions"]):
    formatted_questions["questions"].append({
        "question": question["question"],
        "answer": question["answer"],
        "distractors": question["distractors"],
        "explanation": question["explanation"]
    })

formatted_questions

with open("formatted_quiz_generation_test_output_gemini_gpt4omini.json", "w") as f:
    json.dump(
        formatted_questions,
        f,
        indent=4,
    )

In [13]:
output = await litellm_service.process_async(
    inputs=LiteLLMInput(
        messages=[
            CompletionMessage(
                role=Role.USER,
                content="Hello"
            )
        ],
        model="gemini-2.5-flash",
        temperature=0.0,
        max_completion_tokens=10000,
        top_p=1.0,
        frequency_penalty=0.0,
        n=1,
    )
)
output

LiteLLMOutput(response='Hello! How can I help you today?', completion_tokens=9)